In [1]:
import webbrowser
from aocd.models import Puzzle
puzzle = Puzzle(year=2019, day=9)
webbrowser.open(puzzle.url);

In [2]:
from dataclasses import dataclass
from typing import Callable
import numpy as np
from functools import partial

In [4]:
class ProgramStop(Exception):
    def __init__(self, final_memory):
        super().__init__()
        self.state = final_memory

@dataclass
class OpCode:
    name: str
    code: int
    n_args: int
    func: Callable

def exit(computer):
    raise ProgramStop(computer._memory)
    
def arithmetic(computer, op):
    computer.c = op(computer.a, computer.b)

def input_(computer):
    computer.a = computer.inputs.pop(0)

def output(computer):
    computer.outputs.append(computer.a)

def jump_if(computer, condition):
    if condition(computer.a):
        return computer.b

def arithmethic_comparison(computer, condition):
    computer.c = int(condition(computer.a, computer.b))

def offset_relative_base(computer):
    computer.relative_base += computer.a

code_map = {
    op.code: op for op in [
        OpCode(name="add", code=1, n_args=3, func=partial(arithmetic, op=lambda a, b: a + b)),
        OpCode(name="multiply", code=2, n_args=3, func=partial(arithmetic, op=lambda a, b: a * b)),
        OpCode(name="input", code=3, n_args=1, func=input_),
        OpCode(name="output", code=4, n_args=1, func=output),
        OpCode(name="jump-if-true", code=5, n_args=2, func=partial(jump_if, condition=lambda a: a != 0)),
        OpCode(name="jump-if-false",code=6, n_args=2, func=partial(jump_if, condition=lambda a: a == 0)),
        OpCode(name="less than", code=7, n_args=3, func=partial(arithmethic_comparison, condition=lambda a, b: a < b)),
        OpCode(name="equals", code=8, n_args=3, func=partial(arithmethic_comparison, condition=lambda a, b: a == b)),
        OpCode(name="offset-relbase", code=9, n_args=1, func=offset_relative_base),
        OpCode(name="exit", code=99, n_args=0, func=exit)
    ]
}

def parse_opcode(code: int):
    """
    Parse the opcode and the parameter modes from the given opcode integer
    
    >>> parse_opcode(1002)
    OpCode('multiplay'), [0, 1, 0]
    
    >>> parse_opcode(1107)
    OpCode('less than'), [1, 1, 0]
    """
    instruction = code_map[code % 100]
    modes = [code // 10**(p+2) % 10 for p in range(instruction.n_args)]
    return instruction, modes

In [9]:
np.zeros(len(self.original_memory) * 1000, np.int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
class Computer:
    def __init__(self, memory, inputs=None, debug=False):
        self.original_memory = memory
        self.inputs = [] if inputs is None else inputs
        self.debug = debug
    
    def simulate(self):
        self._memory = self.original_memory.copy()
        self.outputs = []
        try:
            self._ip = 0  # instruction pointer
            self.relative_base = 0
            self._simulate()
            raise ValueError("Program did not terminate and instruction pointer ran out of bounds")
        except ProgramStop as stop:
            if self.debug:
                print("Program Terminated!")
            self._ip = None
            self._final_memory = stop.state
        return self.outputs
    
    def _simulate(self):
        while self._ip < len(self._memory):
            self._instruction, self._param_modes = parse_opcode(self._memory[self._ip])
            self._params = self._memory[self._ip+1 : self._ip+1 + self._instruction.n_args]
            self._log_instruction_call_before()
            new_ip = self._instruction.func(self)
            self._log_instruction_call_after(new_ip)
            if new_ip is None:
                self._ip += 1 + self._instruction.n_args
            else:
                self._ip = new_ip
    
    def start_or_continue(self):
        if hasattr(self, '_ip') and self._ip is not None:
            self._simulate()
        else:
            self.simulate()
    
    def _ensure_enough_memory(self, n):
        while len(self._memory) < n:
            new_mem = np.zeros(len(self._memory) * 2, np.int)
            new_mem[:len(self._memory)] = self._memory
            self._memory = new_mem
            
    
    # easy accessors for the parameters of the current instruction
    def _get_param(self, index):
        assert len(self._params) > index
        if self._param_modes[index] == 0:
            self._ensure_enough_memory(self._params[index])
            return self._memory[self._params[index]]
        elif self._param_modes[index] == 1:
            return self._params[index]
        else:  # mode 2
            self._ensure_enough_memory(self.relative_base + self._params[index])
            return self._memory[self.relative_base + self._params[index]]
            
    
    def _set_param(self, index, value):
        assert len(self._params) > index
        if self._param_modes[index] == 0:
            self._ensure_enough_memory(self._params[index])
            self._memory[self._params[index]] = value
        elif self._param_modes[index] == 2:
            self._ensure_enough_memory(self.relative_base + self._params[index])
            self._memory[self.relative_base + self._params[index]] = value
        else:
            raise ValueError(f"Wrong parameter mode for writing to a value: {self._param_modes[index]}")
    
    @property
    def a(self):
        return self._get_param(0)
    
    @a.setter
    def a(self, val):
        self._set_param(0, val)
    
    @property
    def b(self):
        return self._get_param(1)
    
    @b.setter
    def b(self, val):
        self._set_param(1, val)
    
    @property
    def c(self):
        return self._get_param(2)
    
    @c.setter
    def c(self, val):
        self._set_param(2, val)
    
    def _log_instruction_call_before(self):
        if not self.debug:
            return
        instr = ' '.join(map(str, self._memory[self._ip:self._ip+1 + self._instruction.n_args]))
        params = [f"[{par}]={self._memory[par]}" if m == 0 else str(par) for par, m in zip(self._params, self._param_modes)]
        print(f"{str(self._ip) + '  ':.<6} {instr+'  ':.<20}  {self._instruction.name+'  ':.<15}  {', '.join(params) + '  ':.<40}", end="")
    
    def _log_instruction_call_after(self, new_ip):
        if not self.debug:
            return
        writeable_params = set([par for par, m in zip(self._params, self._param_modes) if m == 0])
        values = [f"mem[{par}] = {self._memory[par]}" for par in writeable_params]
        if new_ip:
            print(f"✓ Instruction Pointer changed: New value {new_ip}")
        elif values:
            print(f"✓ Result: {', '.join(values)}")
        else:
            print("✓ No effect to memory or execution")

## Example

In [18]:
program = np.array([109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99])
assert np.allclose(Computer(program).simulate(), program)

In [19]:
program = np.array([1102,34915192,34915192,7,4,7,99,0])

## Part 1

In [28]:
program = np.array(list(map(int, puzzle.input_data.split(","))))
Computer(program, inputs=[1]).simulate()

[3409270027]

In [23]:
puzzle.answer_a = 3409270027

That's the right answer!  You are one gold star closer to rescuing Santa. [Continue to Part Two]


## Part 2

In [27]:
program = np.array(list(map(int, puzzle.input_data.split(","))))
Computer(program, inputs=[2]).simulate()

[82760]

In [25]:
puzzle.answer_b = 82760

That's the right answer!  You are one gold star closer to rescuing Santa.You have completed Day 9! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
